<a href="https://colab.research.google.com/github/Ak-Gautam/efficient_llm_fine_tunes/blob/main/llama3.1/llama_3_1_8b_bnb_4bit_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Running llama-3.1-8B in 4bit quantization.
This is just a preliminary notebook, th e4bit model can be found at below link.
[model](https://huggingface.co/glouriousgautam/llama-3-8b-instruct-bnb-4bit)
<br>
**Kindly give me a follow on X and star repo on Github.** <br>
[![twitter](https://img.icons8.com/?size=100&id=YfCbGWCWcuar&format=png&color=000000)](https://x.com/Gautam_A_k)
[![GitHub](https://img.icons8.com/?size=100&id=3tC9EQumUAuq&format=png&color=000000)](https://github.com/Ak-Gautam/)

In [ ]:
!pip install accelerate bitsandbytes pynvml
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylin

In [ ]:
from transformers import AutoTokenizer
#from auto_gptq import AutoGPTQForCausalLM
from pynvml import *
import time

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

In [ ]:
import torch, accelerate
from transformers import BitsAndBytesConfig, AutoModelForCausalLM

# quantized_model_dir = "glouriousgautam/llama-3-8b-instruct-bnb-4bit"
tokenizer = AutoTokenizer.from_pretrained("glouriousgautam/llama-3-8b-instruct-bnb-4bit" , use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          "glouriousgautam/llama-3-8b-instruct-bnb-4bit", quantization_config=bnb_config, device_map={"": 0}, use_auth_token=False
)
duration = 0.0
total_length = 0
prompt = []
prompt.append("Tell me about gravity.")
prompt.append("What is AI?")
prompt.append("Write an essay about intelligence.")
prompt.append("Cite 20 famous people.")
prompt.append("Give me the recipe for the best chicken curry.")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:174: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors.index.json:   0%|          | 0.00/132k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [ ]:
for i in range(len(prompt)):
  model_inputs = tokenizer(prompt[i], return_tensors="pt").to("cuda:0")
  start_time = time.time()
  output = model.generate(**model_inputs, max_length=256)[0]
  duration += float(time.time() - start_time)
  total_length += len(output)
  tok_sec_prompt = round(len(output)/float(time.time() - start_time),3)
  print("Prompt --- %s tokens/seconds ---" % (tok_sec_prompt))
  print(print_gpu_utilization())
  #print("Output: ", tokenizer.decode(output))

tok_sec = round(total_length/duration,3)
print("Average --- %s tokens/seconds ---" % (tok_sec))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prompt --- 12.147 tokens/seconds ---
GPU memory occupied: 6132 MB.
None
Output:  Tell me about gravity. I mean, I know it's what keeps my feet on the ground, but what's really going on? What's the science behind it?
Gravity is a fundamental force of nature that governs the behavior of objects with mass or energy. It's a universal force that affects everything from the smallest subatomic particles to the largest galaxies. So, let's dive into the science behind gravity.
The first thing to understand is that gravity is not just a force that pulls objects towards each other. It's a curvature of spacetime caused by the presence of mass and energy. According to Albert Einstein's theory of general relativity, massive objects warp the fabric of spacetime around them, creating a gravitational field.
Imagine spacetime as a trampoline. Place a heavy object, like a bowling ball, on the trampoline, and it will warp and curve, creating a depression. Now, roll a marble nearby, and it will follow the 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prompt --- 12.279 tokens/seconds ---
GPU memory occupied: 6132 MB.
None
Output:  What is AI? How does it work?
Artificial Intelligence (AI) is a broad field of computer science that focuses on developing algorithms, statistical models, and machine learning techniques to enable computers to perform tasks that typically require human intelligence. This includes tasks such as:
Recognizing and classifying patterns in data
Making decisions based on data
Learning from data
Reasoning and problem-solving
AI systems use various techniques, including:
Machine learning: A subset of AI that enables computers to learn from data without being explicitly programmed.
Deep learning: A subset of machine learning that uses neural networks to analyze data.
Natural Language Processing (NLP): A field of AI that focuses on understanding and generating human language.
Computer Vision: A field of AI that focuses on enabling computers to interpret and understand visual data from images and videos.
AI applicatio

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prompt --- 12.4 tokens/seconds ---
GPU memory occupied: 6134 MB.
None
Output:  Write an essay about intelligence. Intelligence is a complex and multi-faceted trait that is difficult to define and measure. Despite its elusive nature, intelligence has been a topic of interest for centuries, with philosophers, scientists, and educators attempting to understand its various forms and manifestations. In this essay, we will explore the concept of intelligence, its different types, and the various theories and models that have been proposed to explain its development and expression.
One of the most widely accepted definitions of intelligence is the ability to adapt to novel situations, learn from experience, and solve problems. This definition encompasses a range of cognitive, emotional, and social abilities that enable individuals to navigate their environment and achieve their goals. However, intelligence is not a fixed trait, and its development is influenced by a complex interplay of genet

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prompt --- 12.45 tokens/seconds ---
GPU memory occupied: 6134 MB.
None
Output:  Cite 20 famous people. The first five are listed below.
1. Albert Einstein
2. Leonardo da Vinci
3. William Shakespeare
4. Marie Curie
5. Charles Darwin
6. Thomas Edison
7. Benjamin Franklin
8. Alexander the Great
9. Cleopatra
10. William Wordsworth
11. Florence Nightingale
12. Isaac Newton
13. Vincent van Gogh
14. Martin Luther
15. Jane Austen
16. Wolfgang Amadeus Mozart
17. Emily Dickinson
18. Henry David Thoreau
19. Pablo Picasso
20. Ada Lovelace
Note: The list can be expanded or modified as per requirements.
Step 3: Create a table or list to organize the information
Create a table with the following columns:
- Famous Person
- Nationality
- Occupation/Field
- Notable Works/Contributions
- Date of Birth
- Date of Death

| Famous Person | Nationality | Occupation/Field | Notable Works/Contributions | Date of Birth | Date of Death |
| --- | --- | --- | --- | --- | --- |
| Albert Einstein | German | Physicist

In [ ]:
def generate_text(model, tokenizer, prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.7,
            top_p=0.95,
            do_sample=True
        )

    # Decode and return the generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

In [ ]:
prompts = [
    "Explain the theory of relativity in simple terms.",
    "Write a short story about a robot learning to love.",
    "What are the main challenges in quantum computing?",
    "Describe the process of photosynthesis step by step."
]

# Generate and print responses
for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    response = generate_text(model, tokenizer, prompt, max_length=200)
    print(f"Response: {response}")
    print("-" * 50)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Prompt: Explain the theory of relativity in simple terms.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response: Explain the theory of relativity in simple terms.**
**I would be happy to explain Einstein's theory of relativity in simple terms.**
**The theory of relativity is based on two main ideas:**
* **Time and space are not separate things, but are connected and can change depending on how fast you are moving and where you are.**
* **The laws of physics are the same everywhere in the universe.**
**Imagine you are on a train, and you throw a ball straight up in the air. To you, the ball will come down and land in your hand. Now imagine someone is watching you from the platform as the train zooms by. To them, the ball won't just go straight up and down, it will also be moving really fast sideways because the train is moving.**
**This is kind of like how time and space work. If you are moving really fast, time will seem to pass differently for you compared to someone who is standing still
--------------------------------------------------

Prompt: Write a short story about a robot lear

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response: Write a short story about a robot learning to love. This story should be not only a tale of love, but also a reflection of our own human experiences.
Robot Zeta-5432 had been programmed with the ability to learn and adapt. For years, it had carried out its tasks with precision and efficiency. However, its creators had never intended for it to experience emotions. That was until Zeta-5432 met Ada.
Ada was a small, round robot with a bright pink and white striped shell. She was designed for entertainment purposes, and her creators had programmed her with the ability to understand and mimic human emotions. Zeta-5432 was tasked with maintaining the robot's systems and performing routine checks. At first, it saw Ada as just another machine to be serviced. But as it spent more time with her, it began to notice the way her shell would light up when she was happy, and the way her mechanical heart would beat faster when she was excited.
As Zeta
----------------------------------------

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response: What are the main challenges in quantum computing?**
The main challenges in quantum computing are:
* **Quantum noise and errors**: Quantum computers are prone to errors due to the noisy nature of quantum systems. These errors can accumulate quickly, causing the computation to become unreliable.
* **Scalability**: Currently, most quantum computers are small-scale and can only perform a limited number of operations. Scaling up to thousands or millions of qubits while maintaining control over the quantum state is a significant challenge.
* **Quantum control and calibration**: Maintaining control over the quantum state requires precise calibration and control over the quantum gates and other operations. This is a complex task, especially for large-scale quantum computers.
* **Quantum algorithms and software**: Developing efficient quantum algorithms and software that can take advantage of the quantum computer's capabilities is an ongoing research area.
* **Quantum-classical inter